In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Papillary_Cell_Carcinoma"
cohort = "GSE19949"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma"
in_cohort_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma/GSE19949"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/GSE19949.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE19949.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/clinical_data/GSE19949.csv"
json_path = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Integrative genome-wide expression profiling identifies three distinct molecular subgroups of renal cell carcinoma with different patient outcome"
!Series_summary	"Background: Renal cell carcinoma (RCC) is characterized by a number of diverse molecular aberrations that differ among individuals. Recent approaches to molecularly classify RCC were based on clinical, pathological as well as on single molecular parameters. As a consequence, gene expression patterns reflecting the sum of genetic aberrations in individual tumors may not have been recognized. In an attempt to uncover such molecular features in RCC, we used a novel, unbiased and integrative approach."
!Series_summary	"Methods: We integrated gene expression data from 97 primary RCCs of different pathologic parameters, 15 RCC metastases as well as 34 cancer cell lines for two-way nonsupervised hierarchical clustering using gene groups suggested by the PANTHER Classification System. We depict

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Dict, Any, Optional

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains genome-wide expression profiling data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary:

# For trait (Kidney Papillary Cell Carcinoma):
# Key 4 contains 'icd-o 3 diagnosis text: papillary renal cell carcinoma'
trait_row = 4

# For age:
# There's no information about age in the sample characteristics
age_row = None

# For gender:
# Key 6 contains gender information
gender_row = 6

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert trait value to binary (0: non-PRCC, 1: PRCC)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if it's papillary renal cell carcinoma
    if 'papillary renal cell carcinoma' in value.lower():
        return 1
    elif 'clear cell' in value.lower() or 'chromophobe' in value.lower() or 'renal cell carcinoma' in value.lower():
        return 0
    else:
        return None

def convert_age(value):
    """This function is included for completeness but won't be used since age data is not available"""
    return None

def convert_gender(value):
    """Convert gender value to binary (0: female, 1: male)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert gender
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Conduct initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Check if trait_row is not None before extracting clinical features
if trait_row is not None:
    # First check for sample_characteristic.csv which is the typical format
    sample_char_path = os.path.join(in_cohort_dir, "sample_characteristic.csv")
    
    # Check if the file exists before trying to load it
    if os.path.exists(sample_char_path):
        clinical_data = pd.read_csv(sample_char_path, header=None)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the selected clinical features
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical features saved to {out_clinical_data_file}")
    else:
        print(f"Clinical data file not found at {sample_char_path}")
        print("Unable to extract clinical features")


Clinical data file not found at ../../input/GEO/Kidney_Papillary_Cell_Carcinoma/GSE19949/sample_characteristic.csv
Unable to extract clinical features


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Integrative genome-wide expression profiling identifies three distinct molecular subgroups of renal cell carcinoma with different patient outcome"
Line 1: !Series_geo_accession	"GSE19949"
Line 2: !Series_status	"Public on Jul 24 2012"
Line 3: !Series_submission_date	"Jan 19 2010"
Line 4: !Series_last_update_date	"Nov 27 2018"
Line 5: !Series_pubmed_id	"22824167"
Line 6: !Series_summary	"Background: Renal cell carcinoma (RCC) is characterized by a number of diverse molecular aberrations that differ among individuals. Recent approaches to molecularly classify RCC were based on clinical, pathological as well as on single molecular parameters. As a consequence, gene expression patterns reflecting the sum of genetic aberrations in individual tumors may not have been recognized. In an attempt to uncover such molecular features in RCC, we used a novel, unbiased and integrative approach."
Line 7: !Series_summary	"Methods: We integrated 

Successfully extracted gene data with 22277 rows
First 20 gene IDs:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Analyze gene identifiers based on their format
# Looking at the first few identifiers: '1007_s_at', '1053_at', '117_at', '121_at', etc.
# These appear to be Affymetrix probe IDs, not standard human gene symbols
# Affymetrix IDs typically have formats like NNN_at, NNN_s_at, etc.
# These will require mapping to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 5177938 rows

Gene annotation preview (first few rows):
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': [nan, nan, nan, nan, nan], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEFINITION=HSPAX8A H.sapi

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for gene mapping
# From the preview, we can see that:
# - 'ID' in gene_annotation contains the probe IDs (like '1007_s_at')
# - 'Gene Symbol' contains the human gene symbols (like 'DDR1')

# 2. Get gene mapping dataframe
mapping_data = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')
print(f"Created gene mapping with {len(mapping_data)} rows")
print("First few rows of mapping data:")
print(preview_df(mapping_data))

# 3. Convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_data)
print(f"Converted gene expression data from probes to genes, resulting in {len(gene_data)} genes")
print("First 20 gene symbols:")
print(gene_data.index[:20])

# Optionally normalize gene symbols to handle synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalizing gene symbols: {len(gene_data)} unique genes")
print("First 20 normalized gene symbols:")
print(gene_data.index[:20])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Created gene mapping with 21248 rows
First few rows of mapping data:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}
Converted gene expression data from probes to genes, resulting in 13046 genes
First 20 gene symbols:
Index(['A2BP1', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAK1',
       'AAMP', 'AANAT', 'AARS', 'AARSD1', 'AASDHPPT', 'AASS', 'AATF', 'AATK',
       'ABAT', 'ABC1', 'ABCA1', 'ABCA11'],
      dtype='object', name='Gene')
After normalizing gene symbols: 12700 unique genes
First 20 normalized gene symbols:
Index(['A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAK1', 'AAMDC',
       'AAMP', 'AANAT', 'AAR2', 'AARS1', 'AARSD1', 'AASDHPPT', 'AASS', 'AATF',
       'AATK', 'ABAT', 'ABCA1', 'ABCA11P'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE19949.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    # Load gene data that was saved in step 6
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Loaded gene data with {gene_data.shape[0]} genes and {gene_data.shape[1]} samples")
    
    # Normalize gene symbols using NCBI Gene database - this was already done in Step 6
    # but we'll ensure it's properly normalized here
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {gene_data.shape[0]} genes")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Extract clinical data directly from the matrix file since it wasn't processed in Step 2
print("\nExtracting clinical data from source...")
try:
    # Get the file paths again to make sure we have them
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Extract background information and clinical data
    background_info, clinical_data = get_background_and_clinical_data(
        matrix_file, 
        prefixes_a=['!Series_title', '!Series_summary', '!Series_overall_design'],
        prefixes_b=['!Sample_geo_accession', '!Sample_characteristics_ch1']
    )
    
    # Process clinical data using trait information from Step 2
    trait_row = 4  # Based on analysis in step 2
    gender_row = 6  # Based on analysis in step 2
    age_row = None  # Based on analysis in step 2
    
    def convert_trait(value):
        """Convert trait value to binary (0: non-PRCC, 1: PRCC)"""
        if pd.isna(value):
            return None
        
        # Extract the value after the colon if present
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        # Check if it's papillary renal cell carcinoma
        if 'papillary renal cell carcinoma' in value.lower():
            return 1
        elif 'clear cell' in value.lower() or 'chromophobe' in value.lower() or 'renal cell carcinoma' in value.lower():
            return 0
        else:
            return None

    def convert_gender(value):
        """Convert gender value to binary (0: female, 1: male)"""
        if pd.isna(value):
            return None
        
        # Extract the value after the colon if present
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        # Convert gender
        if value.lower() == 'female':
            return 0
        elif value.lower() == 'male':
            return 1
        else:
            return None
    
    def convert_age(value):
        """This function is included for completeness but won't be used since age data is not available"""
        return None
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    
    # Check if we have valid trait information
    is_trait_available = trait_row is not None and not selected_clinical_df.empty
    print(f"Trait information available: {is_trait_available}")
    
except Exception as e:
    print(f"Error extracting clinical data: {e}")
    is_trait_available = False
    selected_clinical_df = pd.DataFrame()

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    if is_trait_available and not selected_clinical_df.empty:
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
        print(f"Created linked data with {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    else:
        print("Cannot link data: clinical data is not available")
        linked_data = pd.DataFrame()
        is_trait_available = False
except Exception as e:
    print(f"Error linking clinical and genetic data: {e}")
    is_trait_available = False
    linked_data = pd.DataFrame()

# 4. Handle missing values in the linked data
if is_trait_available and not linked_data.empty:
    print("\nHandling missing values...")
    try:
        # Rename the first column to the trait name for consistency
        if linked_data.columns[0] != trait:
            linked_data = linked_data.rename(columns={linked_data.columns[0]: trait})
        
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        
    # 5. Determine whether the trait and demographic features are biased
    print("\nEvaluating feature bias...")
    try:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Trait bias determination: {is_biased}")
        print(f"Final linked data shape: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error evaluating feature bias: {e}")
        is_biased = True
else:
    print("\nSkipping missing value handling and bias evaluation as linked data is not available")
    is_biased = True

# 6. Validate and save cohort information
print("\nPerforming final validation...")
note = ""
if not is_trait_available:
    note = "Dataset does not contain required trait information"
elif is_biased:
    note = "Dataset has severe bias in the trait distribution"

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. Save the linked data if usable
print(f"\nDataset usability for {trait} association studies: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Final linked data saved to {out_data_file}")
else:
    if note:
        print(f"Reason: {note}")
    else:
        print("Dataset does not meet quality criteria for the specified trait")


Normalizing gene symbols...
Loaded gene data with 12700 genes and 147 samples


After normalization: 12700 genes


Normalized gene data saved to ../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE19949.csv

Extracting clinical data from source...


Clinical data saved to ../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/clinical_data/GSE19949.csv
Clinical data shape: (2, 147)
Trait information available: True

Linking clinical and genetic data...
Created linked data with 147 samples and 12702 features

Handling missing values...


After handling missing values: 113 samples and 12702 features

Evaluating feature bias...
For the feature 'Kidney_Papillary_Cell_Carcinoma', the least common label is '1.0' with 20 occurrences. This represents 17.70% of the dataset.
The distribution of the feature 'Kidney_Papillary_Cell_Carcinoma' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 11 occurrences. This represents 9.73% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Trait bias determination: False
Final linked data shape: 113 samples and 12702 features

Performing final validation...
A new JSON file was created at: ../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/cohort_info.json

Dataset usability for Kidney_Papillary_Cell_Carcinoma association studies: True


Final linked data saved to ../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/GSE19949.csv
